In [29]:
!pip install requests
!pip install rcsbsearchapi


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [3]:
import re
import requests

from rcsbsearchapi.search import SequenceQuery

### Phase 1: Solicit User for UniProt ID and Query/Save the Protein's Similar Sequences

In [15]:
# Define a function that takes a Uniprot ID as input and returns True if the ID exists in Uniprot and False if it doesn't.
def is_uniprot(pid):
    uniprot_api=f"https://www.uniprot.org/uniprot/{pid}.txt"
    response = requests.get(uniprot_api)
    if response.status_code == 200:
        return True
    else:
        pid = input(f"Error: {pid} doesn't exist in Uniprot. Enter a valid Uniprot ID:") # Get UniProt ID from user
        is_uniprot(pid)

# Define a function that takes a Uniprot ID as input and returns a list similar IDs from PubChem BioAssay.
def get_fasta(pid):
    uniprot_fasta_url = f"https://www.uniprot.org/uniprot/{pid}.fasta" # Need to update summary to similarity

    response = requests.get(uniprot_fasta_url)
    if response.status_code == 200:
        # Process the response if request was successful
        fasta = response.text
        return fasta
    else:
        # Handle errors
        print("Error:", response.text)

# Prompt the user for a UniProt ID
pid = input("Enter a Uniprot ID: ")
is_uniprot(pid)
fasta = get_fasta(pid)


In [18]:
# Function to query PDB for sequence similar proteins to the input protein. Returns a list of similar proteins.
def get_similarpid(fasta_sequence):
    search_request = {
        "query": {
            "type": "terminal",
            "service": "sequence",
            "parameters": {
            "evalue_cutoff": 1, # Maybe leave this and identity_cutoff as user input?
            "identity_cutoff": 0.9,
            "sequence_type": "protein",
            "value": fasta_sequence
            }
            },
        "request_options": {
                "scoring_strategy": "sequence",
                "results_verbosity": "verbose",
                "group_by": {
                "aggregation_method": "matching_uniprot_accession",
                "ranking_criteria_type": {
                    "sort_by": "coverage"
            }
        },
        "group_by_return_type": "groups"
    },
    "return_type": "polymer_entity"
}

    # The json parameter in the requests.post automatically converts the Python dictionary to a JSON payload.
    response = requests.post("https://search.rcsb.org/rcsbsearch/v2/query", json=search_request)
    result = response.text
    
    return result

def get_smlmolecules(pdb_id):
    return 0

sequence = re.search(r'(?<=\n)[A-Z\n]+', fasta)
fasta_sequence = sequence.group(0).replace("\n", "")
results = get_similarpid(fasta_sequence)

print(results)

{
  "query_id" : "fb42f4fc-fc26-4de3-8d3b-b27a7353ae0e",
  "result_type" : "polymer_entity",
  "total_count" : 42,
  "group_by_count" : 3,
  "ungrouped_count" : 0,
  "group_by" : {
    "aggregation_method" : "matching_uniprot_accession",
    "ranking_criteria_type" : {
      "sort_by" : "coverage"
    }
  },
  "group_set" : [ {
    "identifier" : "Q92731",
    "score" : 0.8781247735671329,
    "count" : 37,
    "result_set" : [ {
      "identifier" : "1YY4_1",
      "score" : 1.0,
      "services" : [ {
        "service_type" : "sequence",
        "nodes" : [ {
          "node_id" : 13377,
          "original_score" : 529.0,
          "norm_score" : 1.0,
          "match_context" : [ {
            "sequence_identity" : 1.0,
            "evalue" : 3.974E-165,
            "bitscore" : 529,
            "alignment_length" : 268,
            "mismatches" : 0,
            "gaps_opened" : 0,
            "query_beg" : 263,
            "query_end" : 530,
            "subject_beg" : 1,
         

In [36]:
# Clean up the results variable to get a dictionary {UniProt ID: [PDB IDs]}



# FUNCTION: Call the RCSB API for all metadata associated with the PDB ID https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb_id}/1


def fetch_inchi_keys(pdb_id):
    """
    Fetch the InChIKeys for non-polymer entities associated with a given PDB ID.
    
    :param pdb_id: The PDB ID to query.
    :return: The JSON response containing the InChIKeys.
    """
    url = 'https://data.rcsb.org/graphql'
    headers = {'Content-Type': 'application/json'}
    query = """
    query Structure($id: String!) {
      entry(entry_id: $id) {
        rcsb_id
        nonpolymer_entities {
          nonpolymer_comp {
            rcsb_chem_comp_descriptor {
              InChIKey
            }
          }
        }
      }
    }
    """
    variables = {'id': pdb_id}
    response = requests.post(url, headers=headers, json={'query': query, 'variables': variables})
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Query failed to run by returning code of {response.status_code}. {response.text}")


pdb_id = "1YY4"  
result = fetch_inchi_keys(pdb_id)
print(result)

{'data': {'entry': {'rcsb_id': '1YY4', 'nonpolymer_entities': [{'nonpolymer_comp': {'rcsb_chem_comp_descriptor': {'InChIKey': 'YHEHVRSGKUYDON-UHFFFAOYSA-N'}}}]}}}


In [ ]:
# use this section to parse the results from previous code block and store all related UniProt IDs as an array

### Phase 2: Query All Assays Related to all Identified Proteins from PubChem BioAssay

MDIKNSPSSLNSPSSYNCSQSILPLEHGSIYIPSSYVDSHHEYPAMTFYSPAVMNYSIPS
NVTNLEGGPGRQTTSPNVLWPTPGHLSPLVVHRQLSHLYAEPQKSPWCEARSLEHTLPVN
RETLKRKVSGNRCASPVTGPGSKRDAHFCAVCSDYASGYHYGVWSCEGCKAFFKRSIQGH
NDYICPATNQCTIDKNRRKSCQACRLRKCYEVGMVKCGSRRERCGYRLVRRQRSADEQLH
CAGKAKRSGGHAPRVRELLLDALSPEQLVLTLLEAEPPHVLISRPSAPFTEASMMMSLTK
LADKELVHMISWAKKIPGFVELSLFDQVRLLESCWMEVLMMGLMWRSIDHPGKLIFAPDL
VLDRDEGKCVEGILEIFDMLLATTSRFRELKLQHKEYLCVKAMILLNSSMYPLVTATQDA
DSSRKLAHLLNAVTDALVWVIAKSGISSQQQSMRLANLLMLLSHVRHASNKGMEHLLNMK
CKNVVPVYDLLLEMLNAHVLRGCKSSITGSECSPAEDSKSKEGSQNPQSQ



### Phase 3: Find Substances That Were Tested From Assays and Store in Dictionary

### Phase 4: Call Protein Data Bank for Pairs of InChi and UniProt Match 

### Phase 5: Output Remaining Data as a Table